In [1]:
from pathlib import Path
import pandas as pd 
import numpy as np
import re
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.cluster import MiniBatchKMeans
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import DBSCAN

In [3]:
path = Path(r"C:\Users\marta\My drive (marta.infante.baiao@gmail.com) (1)\DTU\Computational Tools\minhashing_results.csv")
data_minhashing = pd.read_csv(path)
df=data_minhashing
df.shape
df.columns

Index(['signature', 'rating'], dtype='str')

In [ ]:
# integer to float
df['signature'] = df['signature'].apply(
    lambda s: np.array(re.findall(r'-?\d+', str(s)), dtype=int)
)
#K-MEANS  
# --- 1. Prepare data ---
# Assuming df['signature'] contains lists or arrays of length 100
X = np.array(df['signature'].tolist())
y = df['rating'].values  # true labels

# --- 2. Run K-Means ---
kmeans = KMeans(n_clusters=5, random_state=42, n_init=10)
clusters = kmeans.fit_predict(X)

In [ ]:
clusters.shape
print(clusters[70:75])
print(np.unique(clusters))

In [ ]:
#EVALUATION using DAVIES-BOULDIN INDEX
from sklearn.metrics import davies_bouldin_score
# --- 3. Evaluate with Davies-Bouldin Index ---
db = davies_bouldin_score(X, clusters)
print(f"Davies-Bouldin Index (DB): {db:.4f}")

In [ ]:
import matplotlib.colors as mcolors
#PLOT in two dimensions using PCA and color by cluster
#prepare data 
X = np.array(df['signature'].tolist())  # shape (n_samples, 100)
# --- 4. Optional: visualize clusters ---
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

n_clusters = 5
cmap = plt.cm.get_cmap('tab10', n_clusters)

bounds = np.arange(0, n_clusters + 1, 1)
norm = mcolors.BoundaryNorm(boundaries=bounds, ncolors=n_clusters)

plt.figure(figsize=(8,6))
scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=clusters, cmap=cmap, norm=norm, alpha=0.7)
plt.title("K-Means clustering of MinHash signatures")
plt.xlabel("PCA 1")
plt.ylabel("PCA 2")
plt.colorbar(label="Cluster ID")
plt.show()

In [ ]:
# Combine into one DataFrame
df_results = pd.DataFrame({
    'rating': y,
    'cluster': clusters
})

# Cross-tabulation to see the distribution of ratings per cluster
cluster_summary = pd.crosstab(df_results['cluster'], df_results['rating'])
print(cluster_summary)
cluster_to_rating = cluster_summary.idxmax(axis=1)
print(cluster_to_rating)